In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import string
import jamspell
import matplotlib.pyplot as plt
from dataset_prepare import ToStrLower, DropDuplicates, FilterByQuantile
from autocompleter import AutoCompleterBaseline, AutoCompleteBySuffix, AutoCompleteByInvertedIndex
from inverted_index import InvertedIndex
from tries import make_trie

In [2]:
col_list = ['wbuser_id', 'UQ', 'cnt', 'lem_uq']
df = pd.read_csv("data/s_hist_mil_preprocessed.csv", usecols=col_list)

## Filter data

In [3]:
# filter out duplicates by wbuser_id and UQ
subset_drop = ["wbuser_id", "UQ"]
drop_duplicates_filter = DropDuplicates(subset_drop)
df = drop_duplicates_filter.apply(df)

DropDuplicates: dataset size 910835 -> 870524


In [4]:
UPPER_QUANTILE = 0.95
LOWER_QUANTILE = 0.001
quantile_filter = FilterByQuantile('cnt', UPPER_QUANTILE)
df = quantile_filter.apply(df)

Lower quantile: 0. Upper quantile: 32576.849999999977
FilterByQuantile: dataset size 870524 -> 826997


In [11]:
uq_to_lemma_map = x['lem_uq'].to_dict()

In [13]:
popularity = df.groupby("UQ").size().reset_index(name='popularity')

### Create inverted index

In [14]:
query_popularity_corpus = zip(popularity['UQ'], popularity['popularity'])

In [15]:
inverted_index = InvertedIndex(query_to_lemma_map=uq_to_lemma_map)
index = inverted_index.process_corpus(query_popularity_corpus)

### Create tree

In [16]:
trie = make_trie(popularity['UQ'].values)

Making database for queries... It length: 511138


### Create spellchecker

In [17]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('data/jamspell_ru_model_subtitles.bin')

True

### Create autocompleter

In [18]:
autocomplete_by_inverted_index = AutoCompleteByInvertedIndex(index, trie, corrector, queries_score_thr=0.1, first_prefix=5, max_candidates=50)

In [33]:
autocomplete_by_inverted_index.query("ноутбук", max_n=10)

[Query indx: 305986. Words: ноутбук. Score: 113. Lemma: ['ноутбук'],
 Query indx: 306019. Words: ноутбук hp. Score: 10. Lemma: ['ноутбук', 'hp'],
 Query indx: 306064. Words: ноутбук игровой. Score: 10. Lemma: ['ноутбук', 'игровой'],
 Query indx: 306004. Words: ноутбук asus. Score: 6. Lemma: ['ноутбук', 'asus'],
 Query indx: 306038. Words: ноутбук lenovo. Score: 6. Lemma: ['ноутбук', 'lenovo'],
 Query indx: 422745. Words: сумка для ноутбука. Score: 52. Lemma: ['сумка', 'для', 'ноутбук'],
 Query indx: 350523. Words: подставка для ноутбука. Score: 24. Lemma: ['подставка', 'для', 'ноутбук'],
 Query indx: 481654. Words: чехол для ноутбука. Score: 18. Lemma: ['чехол', 'для', 'ноутбук'],
 Query indx: 420048. Words: столик для ноутбука. Score: 14. Lemma: ['столик', 'для', 'ноутбук'],
 Query indx: 385327. Words: рюкзак для ноутбука. Score: 9. Lemma: ['рюкзак', 'для', 'ноутбук']]

In [63]:
autocomplete_by_inverted_index.get_candidates("ноут")

[('ноутбука', 247, 0),
 ('ноутбук', 178, 0),
 ('ноут', 101, 0),
 ('ноутбуку', 9, 0),
 ('ноутбуки', 7, 0),
 ('ноутбуков', 6, 0),
 ('ноут8', 2, 0),
 ('ноута', 2, 0),
 ('ноутбукапереходник', 1, 0),
 ('ноутбукаи', 1, 0),
 ('ноутбука/', 1, 0),
 ('ноутбука,', 1, 0),
 ('ноутбук,', 1, 0),
 ('ноутбукт', 1, 0),
 ('ноутбук-трансформер', 1, 0),
 ('ноутбуке', 1, 0),
 ('ноутбуе', 1, 0),
 ('ноутбуеоа', 1, 0),
 ('ноутбуk', 1, 0),
 ('ноутбуу', 1, 0),
 ('ноутбока', 1, 0),
 ('ноутбку', 1, 0),
 ('ноутука', 1, 0),
 ('ноут9', 1, 0),
 ('ноут10', 1, 0)]

In [148]:
autocomplete_by_inverted_index.query("польто", max_n=10)

Original query score for польто: 0.02631578947368418
Fixed word польто -> пальто
Got 100 raw results


[Query indx: 1235887. Words: пальто женское осеннее. Score: 3240. Lemma: ['пальто', 'женский', 'осенний'],
 Query indx: 1235137. Words: пальто женское. Score: 1708. Lemma: ['пальто', 'женский'],
 Query indx: 1233586. Words: пальто. Score: 693. Lemma: ['пальто'],
 Query indx: 1237344. Words: пальто мужское. Score: 446. Lemma: ['пальто', 'мужской'],
 Query indx: 1236312. Words: пальто женское шерстяное. Score: 313. Lemma: ['пальто', 'женский', 'шерстяной'],
 Query indx: 1236561. Words: пальто зимнее женское. Score: 308. Lemma: ['пальто', 'зимний', 'женский'],
 Query indx: 1235410. Words: пальто женское демисезонное. Score: 262. Lemma: ['пальто', 'женский', 'демисезонный'],
 Query indx: 1234768. Words: пальто для девочки. Score: 245. Lemma: ['пальто', 'для', 'девочка'],
 Query indx: 1236476. Words: пальто зимнее. Score: 229. Lemma: ['пальто', 'зимний'],
 Query indx: 1239028. Words: пальто халат. Score: 137. Lemma: ['пальто', 'халат']]

In [149]:
autocomplete_by_inverted_index.query("Игровой ноутбук", max_n=10)

Original query score for игровой: 0.11896243291592123
Original query score for ноутбук: 0.1440138808559861
Got 34 raw results


[Query indx: 705308. Words: игровой ноутбук. Score: 15. Lemma: ['игровой', 'ноутбук'],
 Query indx: 705309. Words: игровой ноутбук 540x. Score: 1. Lemma: ['игровой', 'ноутбук', '540'],
 Query indx: 705310. Words: игровой ноутбук acer. Score: 1. Lemma: ['игровой', 'ноутбук', 'acer'],
 Query indx: 705311. Words: игровой ноутбук hp. Score: 1. Lemma: ['игровой', 'ноутбук', 'hp'],
 Query indx: 705312. Words: игровой ноутбук lenovo. Score: 1. Lemma: ['игровой', 'ноутбук', 'lenovo'],
 Query indx: 705314. Words: игровой ноутбук msi. Score: 1. Lemma: ['игровой', 'ноутбук', 'msi'],
 Query indx: 1170215. Words: ноутбук игровой 1060. Score: 1. Lemma: ['ноутбук', 'игровой', '1060'],
 Query indx: 1170217. Words: ноутбук игровой gtx3050. Score: 1. Lemma: ['ноутбук', 'игровой', 'gtx3050'],
 Query indx: 1077421. Words: мощный игровой ноутбук. Score: 1. Lemma: ['мощный', 'игровой', 'ноутбук'],
 Query indx: 1170220. Words: ноутбук игровой i. Score: 1. Lemma: ['ноутбук', 'игровой', 'i']]

In [155]:
autocomplete_by_inverted_index.query("польто красивое", max_n=10)

Original query score for польто: 0.02631578947368418
Fixed word польто -> пальто
Original query score for красивое: 0.10833333333333328
Got 7 raw results


[Query indx: 1237121. Words: пальто красивое женское. Score: 1. Lemma: ['пальто', 'красивый', 'женский'],
 Query indx: 1237122. Words: пальто красивое необыяное. Score: 1. Lemma: ['пальто', 'красивый', 'необыяной'],
 Query indx: 886826. Words: красивое пальто плащ женское. Score: 1. Lemma: ['красивый', 'пальто', 'плащ', 'женский'],
 Query indx: 886827. Words: красивое пальто стеганое. Score: 1. Lemma: ['красивый', 'пальто', 'стёганый'],
 Query indx: 886806. Words: красивое женское пальто утепленное. Score: 1. Lemma: ['красивый', 'женский', 'пальто', 'утеплить'],
 Query indx: 1236953. Words: пальто капюшон красивое. Score: 1. Lemma: ['пальто', 'капюшон', 'красивый'],
 Query indx: 886813. Words: красивое кашемировое белое пальто. Score: 1. Lemma: ['красивый', 'кашемировый', 'белый', 'пальто']]